In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import random
import string
import collections, itertools
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, 
                         islice, repeat,cycle, chain, product, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop, heapify
from operator    import iand, ior, ilshift, irshift
from bisect      import bisect_left, bisect_right

In [2]:
import itertools

In [3]:
input_path = lambda day : "inputs/day{}.txt".format(day)
input_path_sample = lambda day : "inputs/day{}-sample.txt".format(day)

# Day 1: The Tyranny of the Rocket Equation

In [4]:
fuel = 0
with open(input_path(1), 'r') as f:
    for line in f:
        mass = int(line)
        fuel += mass // 3 - 2
print(fuel)

3160932


In [5]:
def compute_fuel(mass):
    ans = 0
    while True:
        mass = mass // 3 - 2
        if mass <= 0:
            break
        ans += mass
    return ans
assert(compute_fuel(100756) == 50346)
assert(compute_fuel(14) == 2)
assert(compute_fuel(1969) == 966)
fuel = 0
with open(input_path(1), 'r') as f:
    for line in f:
        mass = int(line)
        fuel += compute_fuel(mass)
print(fuel)

4738549


# Day 2: 1202 Program Alarm

In [6]:
def parse_code(code):
    import operator
    n = len(code)
    for i in range(0, n, 4):
        opcode = code[i]
        if opcode == 99:
            break
        else:
            left_idx = code[i + 1]
            right_idx = code[i + 2]
            r_idx = code[i + 3]
            op = operator.add if opcode == 1 else operator.mul
            code[r_idx] = op(code[left_idx], code[right_idx])
    return code
assert(parse_code([1,0,0,0,99]) == [2,0,0,0,99])
assert(parse_code([2,3,0,3,99]) == [2,3,0,6,99])
assert(parse_code([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
assert(parse_code([1,1,1,4,99,5,6,0,99]) == [30,1,1,4,2,5,6,0,99])

In [7]:
with open(input_path(2), 'r') as f:
    memory = list(map(int, f.read().split(',')))
code = memory[:]
code[1] = 12
code[2] = 2
print(parse_code(code)[0])

2890696


In [8]:
def choose_noun_and_verb(target, memory):
    n = len(memory)
    for noun in range(n):
        for verb in range(n):
            code = memory[:]
            code[1] = noun
            code[2] = verb
            if parse_code(code)[0] == target:
                return noun, verb
    return None
target = 19690720
noun, verb = choose_noun_and_verb(target, memory)
print(100 * noun + verb)

8226


# Day 3: Crossed Wires

In [9]:
with open(input_path(3), 'r') as f:
    paths = [tuple(token for token in line.split(',')) for line in f]
path1, path2 = paths

In [10]:
def compute_wire(path):
    moves = dict(zip("LRUD", ((-1, 0), (1, 0), (0, 1), (0, -1))))
    x, y = 0, 0
    wire = {}
    steps = 0
    for item in path:
        d, cnt = item[0], int(item[1:])
        dx, dy = moves[d]
        for i in range(cnt):
            if (x, y) not in wire:
                wire[x, y] = steps
            x, y = x + dx, y + dy
            steps += 1
    return wire

In [11]:
wire1, wire2 = compute_wire(path1), compute_wire(path2)

In [12]:
print(min(abs(x) + abs(y) for x, y in wire1.keys() & wire2.keys() if x != 0 and y != 0))

232


In [13]:
print(min(wire1[key] + wire2[key] for key in wire1.keys() & wire2.keys() if key != (0, 0)))

6084


# Day 4: Secure Container

In [14]:
lo = 245182
hi = 790572
def valid_passwd(pw):
    pre = 10
    has_double = False
    while pw:
        d = pw % 10
        if pre != 10:
            if pre == d:
                has_double = True
            if pre < d:
                return False
        pre = d
        pw //= 10
    return has_double

In [15]:
print(sum(valid_passwd(pw) for pw in range(lo, hi + 1)))

1099


In [16]:
def valid_passwd(pw):
    pw_str = str(pw)
    pre = None
    has_double = False
    for ch, g in itertools.groupby(pw_str):
        cnt = len(list(g))
        if pre and ch < pre:
            return False
        if cnt == 2:
            has_double = True
        pre = ch
    return has_double

In [17]:
print(sum(valid_passwd(pw) for pw in range(lo, hi + 1)))

710


# Day 5: Sunny with a Chance of Asteroids

In [18]:
with open(input_path(5), 'r') as f:
    code = list(map(int, f.read().split(',')))

In [19]:
def parse_code_new(in_code, in_val):
    import operator
    code = in_code[:]
    n = len(code)
    i = 0
    def get(para, mode):
        return code[para] if mode == 0 else para
    while i < n:
        instruct = "{:05d}".format(code[i])
        opcode = int(instruct[-2:])
        left_mode = int(instruct[2])
        right_mode = int(instruct[1])
        r_mode = int(instruct[0])
        if opcode == 99:
            break
        elif opcode == 3:
            code[code[i + 1]] = in_val
            i += 2
        elif opcode == 4:
            print(get(code[i + 1], left_mode))
            i += 2
        elif opcode == 1 or opcode == 2:
            op = operator.mul if opcode == 2 else operator.add
            val = op(get(code[i + 1], left_mode), get(code[i + 2], right_mode))
            code[code[i + 3]] = val
            i += 4
        elif opcode == 5:
            first_para = get(code[i + 1], left_mode)
            if first_para != 0:
                i = get(code[i + 2], right_mode)
            else:
                i += 3
        elif opcode == 6:
            first_para = get(code[i + 1], left_mode)
            if first_para == 0:
                i = get(code[i + 2], right_mode)
            else:
                i += 3
        elif opcode == 7:
            if get(code[i + 1], left_mode) < get(code[i + 2], right_mode):
                code[code[i + 3]] = 1
            else:
                code[code[i + 3]] = 0
            i += 4
        elif opcode == 8:
            if get(code[i + 1], left_mode) == get(code[i + 2], right_mode):
                code[code[i + 3]] = 1
            else:
                code[code[i + 3]] = 0
            i += 4
        else:
            assert(False)
    return code

In [20]:
parse_code_new(code, in_val=1);

0
0
0
0
0
0
0
0
0
6731945


In [21]:
test_in = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
parse_code_new(test_in, 7);

999


In [22]:
parse_code_new(code, in_val=5);

9571668


# Day 6: Universal Orbit Map

In [23]:
def count_orbits(infile):
    import collections
    tree = collections.defaultdict(list)
    with open(infile, 'r') as f:
        for line in f:
            u, v = line.strip().split(')')
            tree[u].append(v)
    norbits = 0
    def dfs(node, depth):
        nonlocal norbits
        norbits += depth
        for child in tree[node]:
            dfs(child, depth + 1)
    dfs('COM', 0)
    return norbits

In [24]:
# print(count_orbits(input_path_sample(6)))
assert(count_orbits(input_path_sample(6)) == 42)

In [25]:
print(count_orbits(input_path(6)))

301100


In [26]:
def find_path(infile, target="SAN"):
    import collections
    tree = collections.defaultdict(list)
    with open(infile, 'r') as f:
        for line in f:
            u, v = line.strip().split(')')
            tree[u].append(v)
            tree[v].append(u)
    frontier = [("YOU", None, 0)]
    while frontier:
        node, parent, depth = frontier.pop()
        if node == target:
            return depth - 2
        for nei in tree[node]:
            if nei != parent:
                frontier.append((nei, node, depth + 1))

In [27]:
print(find_path(input_path(6)))

547
